## Step 1

In [ ]:
import os
import shutil
import csv

# Nel primo step del progetto bisogna creare un codice che iteri all'interno della cartella files, stampando le informazione nel formato richiesto
# all'interno dell'assignment, tenendo traccia degli spostamenti del file all'interno delle rispettive cartelle all'interno di un file csv di recap

# Dopo aver importato i moduli che serviranno, per prima cosa definisco una funzione che, quando chiamata, faccia quanto detto prima e inoltre,
# qualora il file non sia già all'interno della sottocartella corrispondente, lo sposti con la funzione shutil.move

def elaborate(file, directory, filetype):
    
    # Stampo le informazioni del file secondo il formato richiesto
    print("{} type:{} size:{}B".format(file.split(".")[0], filetype, os.path.getsize(os.path.join(directory,file))))
    
    # Aggiorno il file csv di recap
    new_writer.writerow([file.split(".")[0], filetype, os.path.getsize(os.path.join(directory, file))])
    
    # Sposto il file nella sottocartella corrispondente, qualora non sia già al suo interno
    if not os.path.isfile(os.path.join(directory, filetype, file)):
        shutil.move(os.path.join(directory, file), os.path.join(directory, filetype))    

In [ ]:
# Per evitare di usare path assoluti, procedo con i path relativi, cambiando la directory all'interno della quale sto operando

os.chdir("../files/") 
directory = os.getcwd() # per rendere il tutto più leggibili, definisco le nuove variabili
files = os.listdir(directory)

# Procedo con un for loop per creare le sottocartelle qualora non esistessero all'interno della working directory attuale
for dirname in ["audio", "images", "docs"]:
    if not os.path.isdir(os.path.join(directory, dirname)):
        os.makedirs(os.path.join(directory, dirname))
        
# Creo delle liste con tutti i formati file accettati
audio_ext = [".mp3"]
images_ext = [".jpg", ".jpeg", ".png"]
docs_ext = [".txt", ".odt"]

# Se il file di recap csv non esiste già lo creo
if not os.path.isfile(os.path.join(directory, "recap.csv")):
    with open(os.path.join(directory, "recap.csv"), "w", newline="") as my_csv:
        csv_writer = csv.writer(my_csv)
        csv_writer.writerow(["name", "type", "size(B)"]) # qui importo gli header del file di recap

# Ora bisogna aprire il file csv e per evitare di sovrascrivere le informazioni (cosa che succederebbe usando "w") utilizzo il comando "a", che sta per "append"
open_csv = open(os.path.join(directory, "recap.csv"), "a", newline="")
new_writer = csv.writer(open_csv)

# e ora elaboro i file nella directory grazie alla funzione elaborate

for file in files:
    name, extension = os.path.splitext(file)
    
    if extension in audio_ext:
        elaborate(file, directory, "audio")
    elif extension in images_ext:
        elaborate(file, directory, "images")
    elif extension in docs_ext:
        elaborate(file, directory, "docs")
    else:
        print("Formato file non supportato")
        
open_csv.close() # alla fine chiudo il file csv aperto

## Step 3

In [ ]:
# Nel terzo step bisogna lavorare sulle immagini presenti nella sottocartella images
# Prima importiamo i moduli che ci servono, rispettivamente Image (dalla libreria PIL) e NumPy, assieme al modulo tabulate per la tabella

import os
import numpy as np
from PIL import Image
from tabulate import tabulate

In [ ]:
# Adesso mi devo concentrare sulla sottocartella images, quindi imposto le nuove variabili
new_dir = os.path.join(directory, "images")
images = os.listdir(new_dir)

#Intanto apro una lista vuota che mi servirà per inserire le informazioni delle immagini nella tabella
table = []

# Utilizzo ancora un for loop che operi all'interno della sottocartella su ogni file presente
for image in images:
    name_img = image.split(".")[0] # nome dell'immagine senza la sua estensione
    read_img = Image.open(os.path.join(new_dir, image)) # apro l'immagine
    np_image = np.array(read_img) # e la trasformo un ndarray
    height = np_image.shape[0] # con la funzione shape posso ottenere l'altezza (height)
    width = np_image.shape[1] # e l'ampiezza (width) dell'immagine
    R, G, B, grayscale, ALPHA=0, 0, 0, 0, 0
    # In particolar modo, dobbiamo fare attenzione alla shape delle immagini e al numero di dimensioni. Nella grayscale avremo due dimensioni, con profondità
    # (il terzo valore di shape) uguale a 1.
    # Una RGB ha 3 dimensioni e 3 canali di colore: ogni pixel avrà dunque tre valori da 0 a 255
    if np_image.ndim == 3 and np_image.shape[2] == 3:
        R, G, B = np.mean(np_image, axis=(0,1))
    elif np_image.ndim == 3 and np_image.shape[2] == 4: #Una RGBA ha 4 canali di colore e si distinguono per il canale ALPHA
        R, G, B, ALPHA = np.mean(np_image, axis=(0,1))
    else: # Altrimenti è una normale grayscale, dove ogni pixel ha solo un valore da 0 a 255
        grayscale = np.mean(np_image)
    # alla fine del loop, inserisco i dati all'interno della lista precedentemente creata    
    table += [[name_img, height, width, grayscale, R, G, B, ALPHA]]
    

In [ ]:
# Con il modulo tabulate inserisco le informazioni raccolte all'interno di una tabella, inserendo gli headers, il formato
# e i decimali che voglio visualizzare all'interno delle celle
print(tabulate(table, headers=["name", "height", "width", "grayscale", "R", "G", "B", "ALPHA"], tablefmt="fancy_grid", floatfmt=".2f"))